This notebook applies a stateful LSTM model using PyTorch & Lightning, to get multi-step quantile energy consumption forecasts. 
\
It also performs the necessary data handling & preprocessing steps to get input & output sequences for the LSTM model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import lightning as L
import optuna
import warnings

from lightning.pytorch.callbacks import Callback
#from pytorch_forecasting.metrics.quantile import QuantileLoss
from optuna.integration import PyTorchLightningPruningCallback
from sklearn.metrics import mean_absolute_error as mae, mean_absolute_percentage_error as mape, root_mean_squared_log_error as rmsle, mean_pinball_loss as pinball

In [ ]:
random_seed = 1923

In [ ]:
# Set Torch settings
torch.set_default_dtype(torch.float32)
torch.set_float32_matmul_precision('medium')
L.seed_everything(random_seed, workers = True)
warnings.filterwarnings("ignore", ".*does not have many workers.*")

In [ ]:
# Plot settings
plt.rcParams["figure.autolayout"] = True
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

In [ ]:
output_dir = "./OutputData/"

In [ ]:
df = pd.read_csv(output_dir + "train_data.csv")
df["time"] = pd.to_datetime(df["time"])

In [ ]:
df

## Data prep: Getting input & output sequences

We will create a "shifted dataset" where each row at time T contains the following columns:
- Target value at T (consumption lead T+1),
- Past value at T (consumption lag T-2),
- Time covariates at T.

In [ ]:
# Get shifted dataset, drop last row due to unknown target
df_shifted = df.copy()
df_shifted["consumption_MWh"] = df_shifted.consumption_MWh.shift(-1)
df_shifted = df_shifted.dropna()

In [ ]:
df_shifted

Every input sequence will be the last 72 hours before 16:00, and every output sequence will be the next 32 hours after 16:00.

In [ ]:
# Define fixed model parameters
n_steps = len(df_shifted) 
input_length = 72 # T-L to T hours as input
input_dims = 10 # Consumption lag 2, trend, 6 cyclical columns + 2 Fourier pairs as features
output_length = 32 # We are only interested in predicting T+8 to 32, but we have to predict from T+1 because we need hidden states at each time step.
horizon_start = 0 # Output step to start loss calculation from
quantiles = [0.025, 0.5, 0.975] # Quantiles to predict

In [ ]:
# Find the index of the first 16:00 row in the data, where the index is bigger than input_length - 1. This will be the first T.
first_t = df_shifted.loc[(df_shifted.time.dt.hour == 16) & (df_shifted.index >= input_length - 1)].index.values[0]

In [ ]:
first_t 

In [ ]:
# Find the index of the last 16:00 row in the data, with 32 time steps after it. This will be the last T.
last_t = df_shifted.loc[(df_shifted.time.dt.hour == 16) & (df_shifted.index + output_length - 1 <= df.index.values[-1])].index.values[-2]

In [ ]:
last_t 

Each input sequence will contain the following:
- Consumption lag 2 at [T - input_length, T] (past targets),
- Time covariates at [T - input_length + 1, T+1 ] (future known covariates).

In [ ]:
# One input sequence
input_seq = pd.concat([
    df_shifted.iloc[(first_t - input_length):first_t, 0], # Time
    df_shifted.iloc[(first_t - input_length):first_t, 2], # Past target
    df_shifted.shift(-1).iloc[(first_t - input_length):(first_t), 3:] # Future known covariates
], axis = 1)

Each output sequence will contain the following:
- Consumption at [T+1, T + output_length] (future targets),
- Time covariates at [T+2, T + output_length + 1] (future known covariates for prediction steps after T+1)

In [ ]:
# One output sequence
output_seq = pd.concat([
    df_shifted.iloc[first_t:(first_t + output_length), 0], # Time 
    df_shifted.iloc[first_t:(first_t + output_length), 1], # Future target
    df_shifted.shift(-1).iloc[first_t:(first_t + output_length), 3:] # Future known covariates
], axis = 1)

In [ ]:
input_seq.iloc[-5:]

In [ ]:
output_seq[:5]

In [ ]:
df_shifted.iloc[81:90]

In the input sequence, we pair every past target value at T with the future covariates at T+1.
\
In the output sequence, we pair every future target value at T+1 with the future covariates at T+2.
\
This is because the future target at T+1 and the future covariates at T+2 will be the past target & future covariates in the next prediction step. 
\
LSTMs and RNNs can only forecast 1 step at a time as they need hidden & cell states from the previous step. 
\
For validation & prediciton steps, we will replace the future targets after T+1 with predicitons from the previous step, as these will be unknown values at real prediciton time.
\
During training, we still use the real target values for all prediciton steps "in hindsight", as training with predictions as the target may mislead the model. In a real life scenario, we'd have the "hindsight" values available in the historic data, just like we do here.

In [ ]:
n_sequences = (last_t - first_t) // 24 + 1 # Number of 16:00 rows followed by a sufficient input / output sequence
print("Number of possible sequences: " + f"{n_sequences}")

In [ ]:
# Get all sequences
for t in range(first_t, last_t + 1, 24):

    # Get input sequence
    new_input = pd.concat([
        df_shifted.iloc[(t - input_length):t, 0], # Time
        df_shifted.iloc[(t - input_length):t, 2], # Past target
        df_shifted.shift(-1).iloc[(t - input_length):t, 3:] # Future known covariates
        ], axis = 1)
    new_input = new_input.set_index("time")

    # Get output sequence
    new_output = pd.concat([
        df_shifted.iloc[t:(t + output_length), 0], # Time 
        df_shifted.iloc[t:(t + output_length), 1], # Future target
        df_shifted.shift(-1).iloc[t:(t + output_length), 3:] # Future known covariates
        ], axis = 1)
    new_output = new_output.set_index("time")

    if t == first_t:

        # Initialize lists of sequences
        input_sequences = [new_input]
        output_sequences = [new_output]
        
    else:
        # Concatenate to arrays of sequences
        input_sequences.append(new_input)
        output_sequences.append(new_output)


In [ ]:
len(input_sequences)

In [ ]:
len(output_sequences)

## Preprocessing: Custom scaler, Torch datasets & dataloaders

In [ ]:
# Get indices for train - val - test split
sixty_percent = int(len(input_sequences) * 0.6)
twenty_percent = int(len(input_sequences) * 0.2)
train_end = sixty_percent
val_end = sixty_percent + twenty_percent

In a stateful LSTM, the last timestep's hidden & cell state for observation 1 in batch 1 will be used as the initial hidden & cell state for observation 1 in batch 2.
\
Because of this, we need batches of constant size through training, val. and testing data. We'll divide the lengths of each data fold with the batch size, drop the remainder from the start of the training fold & the end of the val. & test folds.

In [ ]:
# Perform train - val - test split
batch_size = 64

# Training data at validation step
tr_input, tr_output = input_sequences[0:train_end], output_sequences[0:train_end]
remainder = len(tr_input) % batch_size
tr_input, tr_output = tr_input[remainder:], tr_output[remainder:]

# Training data at testing step
train_input, train_output = input_sequences[0:val_end], output_sequences[0:val_end]
remainder = len(train_input) % batch_size
train_input, train_output = train_input[remainder:], train_output[remainder:]

# Validation data
val_input, val_output = input_sequences[train_end:val_end], output_sequences[train_end:val_end]
remainder = len(val_input) % batch_size
val_input, val_output = val_input[:-remainder], val_output[:-remainder]

# Testing data
test_input, test_output = input_sequences[val_end:], output_sequences[val_end:]
remainder = len(test_input) % batch_size
test_input, test_output = test_input[:-remainder], test_output[:-remainder]

In [ ]:
test_input[0]

In [ ]:
# First prediction point in testing data
test_output[0].index[0]

We have to scale the past consumption & trend values in the input sequences, and the future consumption & trend values in the output sequences, because they'll be the past values as the forecast horizon expands.
\
We also need the ability to backtransform the network's final predictions accordingly.

In [ ]:
# Define scaling class for sequence data
class sequence_scaler:
    """
    Takes in lists of dataframes, where each dataframe is an input or output sequence.

    Returns scaled 3D numpy array of shape (observations, timesteps, features).
    Can also backtransform scaled predictions.
    """

    def __init__(self, feature_range = (-1, 1)):
        self.lower = feature_range[0]
        self.upper = feature_range[1]

    def fit(self, input_df, output_df):

        # Get input & output sequences as 3D arrays
        # The time index will be skipped, yielding shape (N, seq_length, seq_dims)
        input = np.stack(input_df, axis = 0)
        output = np.stack(output_df, axis = 0)

        # Get number of dimensions
        self.num_dimensions = input.shape[2]
        
        # Extract & save minimum, maximum for each dimension
        dimensions_mini = []
        dimensions_maxi = []
        for dimension in range(0, self.num_dimensions):
            min = np.min([
                np.min(input[:, :, dimension]),
                np.min(output[:, :, dimension])
            ])
            dimensions_mini.append(min)

            max = np.max([
                np.max(input[:, :, dimension]),
                np.max(output[:, :, dimension])
            ])
            dimensions_maxi.append(max)

        self.dimensions_mini = dimensions_mini
        self.dimensions_maxi = dimensions_maxi

    def transform(self, scale_df):

        # Get sequence as 3D arrays
        scale_array = np.stack(scale_df, axis = 0)

        # Initialize list of scaled dimensions
        scaled_dimensions = []

        # Scale each dimension & append to list
        for dimension in range(0, self.num_dimensions):
            values = scale_array[:, :, dimension]
            min = self.dimensions_mini[dimension]
            max = self.dimensions_maxi[dimension]
            std = (values - min) / (max - min)
            scaled = std * (self.upper - self.lower) + self.lower
            scaled_dimensions.append(scaled)

        # Stack over 3rd axis & return
        return np.stack(scaled_dimensions, axis = 2)

    def backtransform_preds(self, preds_array, fitted_preds_dim = 0):

        # Get n. of predicted quantiles to backtransform
        n_quantiles = preds_array.shape[-1]

        # Get the fitted mini & maxi for predictions
        min = self.dimensions_mini[fitted_preds_dim] 
        max = self.dimensions_maxi[fitted_preds_dim]

        # Initialize list of backtransformed quantiles
        backtrafo_quantiles = []

        # Backtransform each quantile & append to list
        for quantile in range(0, n_quantiles):
            scaled = preds_array[:, :, quantile]
            std = (scaled - self.lower) / (self.upper - self.lower)
            values = std * (max - min) + min
            backtrafo_quantiles.append(values)
            
        # Stack over 3rd axis & return
        return np.stack(backtrafo_quantiles, axis = 2)

In [ ]:
# Define Torch dataset class
class SequenceDataset(torch.utils.data.Dataset):
    """
    Simply takes in the input & output sequences as 3D arrays and returns them as Torch tensors.
    """

    # Store preprocessed input & output sequences
    def __init__(self, input_seq, output_seq): 
        self.input_seq = torch.tensor(input_seq, dtype = torch.float32) # Store input sequences
        self.output_seq = torch.tensor(output_seq, dtype = torch.float32) # Store output sequences
  
    # Return data length  
    def __len__(self):
        return len(self.input_seq) 
  
    # Return a pair of input & output sequences
    def __getitem__(self, idx):
        return self.input_seq[idx], self.output_seq[idx]

In [ ]:
# Scale validation data & create Torch datasets
scaler_val = sequence_scaler()
_ = scaler_val.fit(tr_input, tr_output)
tr_data = SequenceDataset(scaler_val.transform(tr_input), scaler_val.transform(tr_output))
val_data = SequenceDataset(scaler_val.transform(val_input), scaler_val.transform(val_output))

# Scale testing data & create Torch datasets
scaler_test = sequence_scaler()
_ = scaler_test.fit(train_input, train_output)
train_data = SequenceDataset(scaler_test.transform(train_input), scaler_test.transform(train_output))
test_data = SequenceDataset(scaler_test.transform(test_input), scaler_test.transform(test_output))

In [ ]:
# Create Torch dataloaders
num_workers = 0
shuffle = False

# Training data at validation step
tr_loader = torch.utils.data.DataLoader(
    tr_data, batch_size = batch_size, num_workers = num_workers, shuffle = shuffle)

# Training data at testing step
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size = batch_size, num_workers = num_workers, shuffle = shuffle)

# Validation & testing data
val_loader = torch.utils.data.DataLoader(
    val_data, batch_size = batch_size, num_workers = num_workers, shuffle = shuffle)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size = batch_size, num_workers = num_workers, shuffle = shuffle)

## Model definition

In [ ]:
# Define QuantileLoss, because pytorch_forecasting installation raises issues with Python 3.12
class QuantileLoss:
    """
    Takes in targets of shape (...),
    predictions of shape (..., n_quantiles),
    quantiles list.
    
    Returns unreduced quantile loss tensor of shape (..., n_quantiles),
    where each value is quantile loss * 2 (equal to the MAE for q = 0.5).
    
    Implemented from pytorch_forecasting.metrics.quantile.QuantileLoss.
    """

    def __init__(self, quantiles):
        self.quantiles = quantiles

    def loss(self, pred, target):
        
        quantile_losses = []
        for i, q in enumerate(self.quantiles):
            error = target - pred[..., i]
            quantile_error = torch.max(
                (q - 1) * error,
                q * error
            ).unsqueeze(-1)
            quantile_losses.append(quantile_error)

        quantile_losses = torch.cat(quantile_losses, dim = 2)
        return quantile_losses

In [ ]:
# Define model class
class StatefulQuantileLSTM(L.LightningModule):
    """
    Stateful LSTM forecasting model, returns quantile predictions.
    Input & output sequences are 3D tensors of shape (batch_size, timesteps, features).
    Hidden & cell states are retained & passed forward across training & inference batches.
    """

    # Initialize model
    def __init__(self, hyperparams_dict):
        
         # Delegate function to parent class
        super().__init__() 
        
        # Save external hyperparameters so they are available when loading saved models
        self.save_hyperparameters(logger = False) 

        # Define hyperparameters
        self.output_length = hyperparams_dict["output_length"] # Length of output sequence
        self.input_size = hyperparams_dict["input_size"] # Number of features (network inputs)
        self.horizon_start = hyperparams_dict["horizon_start"] # Start of the forecast horizon relevant for loss computing
        self.quantiles = hyperparams_dict["quantiles"] # Provide as list of floats: [0.025, 0.5, 0.975]
        self.learning_rate = hyperparams_dict["learning_rate"]
        self.lr_decay = hyperparams_dict["lr_decay"]
        self.num_layers = hyperparams_dict["num_layers"] # Number of layers in the LSTM block
        self.hidden_size = hyperparams_dict["hidden_size"] # Number of units in each LSTM block = LSTM block output size
        self.dropout_rate = hyperparams_dict["dropout_rate"]

        # Define architecture
        
        # LSTM input: input, (prev_hidden_states, prev_cell_states)
        # Shapes: (N, input_length, input_size), ((num_layers, N, hidden_size), (num_layers, N, hidden_size))
        self.lstm = torch.nn.LSTM(
            input_size = self.input_size,
            hidden_size = self.hidden_size,
            num_layers = self.num_layers,
            batch_first = True
        )
        # LSTM output: output, (last_hidden_states, last_cell_states)
        # Shapes: (N, input_length, hidden_size), ((num_layers, N, hidden_size), (num_layers, N, hidden_size))
        # "output" has the last layer's output / final hidden state for each timestep in the input sequence.
        # The tuple of hidden & cell states have the last timestep's hidden & cell states for each LSTM layer.

        # Output layer input: LSTM output, shape (N, 1, hidden_size)
        # The final hidden state output for the last timestep in the input sequence.
        self.output_layer = torch.nn.Linear(
            in_features = self.hidden_size,
            out_features = len(self.quantiles)
        )
        # Output layer output: Quantile predictions, shape (N, n_quantiles)

        # Loss function: Quantile loss
        self.loss = QuantileLoss(quantiles = self.quantiles)
        self._median_quantile = np.median(self.quantiles)
        self._median_quantile_idx = self.quantiles.index(self._median_quantile)

        # Initialize hidden & cell state containers for statefulness
        self._last_hiddens_train = None
        self._last_cells_train = None
        self._final_hiddens_train = None
        self._final_cells_train = None

    # Define forward propagation
    # Pass prev_states as tuple (prev_hidden_states, prev_cell_states)
    def forward(self, input_chunk, prev_states = None): 

        # Pass inputs through LSTMs
        # If prev_states is not passed, they are automatically initialized as zeroes
        if prev_states == None:
            lstm_output, (last_hidden_states, last_cell_states) = self.lstm(input_chunk)
        else: 
            lstm_output, (last_hidden_states, last_cell_states) = self.lstm(input_chunk, prev_states)

        # Pass final LSTM output through output layer. Keep in mind this is the last layer's hidden state
        # output for the last timestep in the input sequence.
        preds = self.output_layer(lstm_output[:, -1, :])

        return last_hidden_states, last_cell_states, preds

    # Retain the computational graphs across backprop steps, so backprop across time can be performed
    # across batches. Demands more GPU memory.
    def backward(self, loss):

        # Free the computational graph on the last step of an epoch.
        # If this is not done, GPU memory will fill up, even after the model itself is deleted.
        if self.trainer.is_last_batch:
            loss.backward(retain_graph = False)

        # Retain the computational graph on all steps except last in an epoch.
        else:
            loss.backward(retain_graph = True)

    # Define training step
    def training_step(self, batch, batch_idx):

        # Initialize variables to record horizon, hidden & cell states, predictions
        h = 0
        prev_hiddens = []
        prev_cells = []
        batch_preds = []

        # Get inputs & outputs for first forecast step
        input_sequences, output_sequences = batch
        input_seq = input_sequences # Inputs of the forecast step 0. (N, input_length, input_size) 
        output_seq = output_sequences[:, 0, :] # Target & future covars of forecast step 0. Needed for later forecast steps. (N, input_size)

        # Perform training & recording for first forecast step
        # If a hidden & cell state is retained from the previous batch, use it. This will be the case for all batches except the first in an epoch.
        if self._last_hiddens_train == None:
            last_hidden_states, last_cell_states, preds = self.forward(input_seq)
        else:
            last_hidden_states, last_cell_states, preds = self.forward(
                input_seq, 
                prev_states = (self._last_hiddens_train, self._last_cells_train)
            )

        prev_hiddens.append(last_hidden_states) # 1-dimensional list. Each element has shape (num_layers, N, hidden_size)
        prev_cells.append(last_cell_states) # 1-dimensional list. Each element has shape (num_layers, N, hidden_size)
        batch_preds.append(preds) # 1-dimensional list. Each element has shape (N, n_quantiles)
        h += 1

        # Perform training & recording for remaining forecast steps
        while h < self.output_length:

            # Get inputs & outputs for forecast step h: 
            input_seq = torch.cat((
                input_seq[:, 1:, :], # Inputs of the previous forecast step, with the first row dropped. (N, input_length - 1, input_size)
                output_seq.unsqueeze(1) # Target & future covars of previous forecast step, the last row of the new input. (N, 1, input_size)
            ), dim = 1)
            
            output_seq = output_sequences[:, h, :] # Target & covars. of forecast step h. Needed for later forecast steps. (N, input_size)

            # Perform training & recording for forecast step h:
            last_hidden_states, last_cell_states, preds = self.forward(
                input_seq, 
                prev_states = (prev_hiddens[h-1], prev_cells[h-1])
            )
            prev_hiddens.append(last_hidden_states)
            prev_cells.append(last_cell_states)
            batch_preds.append(preds)
            h += 1

        # Calculate quantile loss for all forecast steps starting from the horizon
        # We're only interested in predicting from T+8 to T+32, but predictions at T+1 will depend on predictions at T,
        # so calculating loss over all h steps is probably ideal (horizon_start = 0). The code below supports either method.
        preds_horizon = batch_preds[self.horizon_start:] # List length (output_length - horizon_start). Each element has shape (N, n_quantiles).
        preds_horizon = torch.stack(preds_horizon, dim = 1) # Shape (N, output_length - horizon_start, n_quantiles)
        targets_horizon = output_sequences[:, self.horizon_start:, 0] # Target values from horizon to end of sequence. Shape(N, output_length - horizon_start)
        loss = self.loss.loss(preds_horizon, targets_horizon) # Quantile losses for each batch & timestep. Shape (N, output_length - horizon_start, n_quantiles)

        # Reduce the quantile loss
        # A lot of room for experimentation here. Below is the typical application for RNNs. See: 
        # (https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks#overview)
        loss_reduced = loss.mean(dim = 2) # Average over quantiles. Yields (N, output_length - horizon_start)
        loss_reduced = loss_reduced.sum(dim = 1) # Sum over forecast steps. Yields (N)
        loss_reduced = loss_reduced.mean() # Average over batches. Yields scalar loss for backpropagation.
        
        # Log the training loss
        self.log("train_loss", loss_reduced, on_step = True, on_epoch = True, prog_bar = True, logger = False)

        # Update last hidden & cell states from training (for within-epoch use)
        self._last_hiddens_train = prev_hiddens[-1]
        self._last_cells_train = prev_cells[-1]

        # Update final hidden & cell states from training (for inference)
        self._final_hiddens_train = prev_hiddens[-1]
        self._final_cells_train = prev_cells[-1]

        return loss_reduced

    # When a training epoch ends, flush the last hidden & cell states.
    # Final hidden & cell states remain for inference.
    def on_train_epoch_end(self):
        self._last_hiddens_train = None
        self._last_cells_train = None

    # Method to flush the final hidden & cell states left from training, if desired
    def reset_states(self):
        self._final_hiddens_train = None
        self._final_cells_train = None

    # Define validation_step
    def validation_step(self, batch, batch_idx):

        # Initialize variables to record horizon, hidden & cell states, predictions
        h = 0
        prev_hiddens = []
        prev_cells = []
        batch_preds = []

        # Get inputs & outputs for first forecast step
        input_sequences, output_sequences = batch
        input_seq = input_sequences # Inputs of the forecast step 0. (N, input_length, input_size) 
        output_seq = output_sequences[:, 0, 1:] # Future covars of forecast step 0. Needed for later forecast steps. (N, input_size - 1)

        # Perform validation & recording for first forecast step
        # If a hidden & cell state is retained from training, use it.
        if self._final_hiddens_train == None:
            last_hidden_states, last_cell_states, preds = self.forward(input_seq)
        else:
            last_hidden_states, last_cell_states, preds = self.forward(
                input_seq, 
                prev_states = (self._final_hiddens_train, self._final_cells_train)
            )

        prev_hiddens.append(last_hidden_states) # 1-dimensional list. Each element has shape (num_layers, N, hidden_size)
        prev_cells.append(last_cell_states) # 1-dimensional list. Each element has shape (num_layers, N, hidden_size)
        batch_preds.append(preds) # 1-dimensional list. Each element has shape (N, n_quantiles)
        h += 1

        # Perform validation & recording for remaining forecast steps
        while h < self.output_length:

            # Get inputs & outputs for forecast step h:
            output_seq = torch.cat((
                batch_preds[h-1][:, self._median_quantile_idx].unsqueeze(1), # Point prediction of forecast step 0. (N, 1)
                output_seq # Future covars of forecast step 0. (N, input_size - 1)
            ), dim = 1)
            
            input_seq = torch.cat((
                input_seq[:, 1:, :], # Inputs of the previous forecast step, with the first row dropped. (N, input_length - 1, input_size)
                output_seq.unsqueeze(1) # Prediction & future covars of previous forecast step, the last row of the new input. (N, 1, input_size)
            ), dim = 1)
            
            output_seq = output_sequences[:, h, 1:] # Future covars. of forecast step h. Needed for later forecast steps. (N, input_size-1)

            # Perform training & recording for forecast step h:
            last_hidden_states, last_cell_states, preds = self.forward(
                input_seq, 
                prev_states = (prev_hiddens[h-1], prev_cells[h-1])
            )
            prev_hiddens.append(last_hidden_states)
            prev_cells.append(last_cell_states)
            batch_preds.append(preds)
            h += 1

        # Calculate loss for forecast steps starting from horizon
        preds_horizon = batch_preds[self.horizon_start:] # List length (output_length - horizon_start). Each element has shape (N, n_quantiles).
        preds_horizon = torch.stack(preds_horizon, dim = 1) # Shape (N, output_length - horizon_start, n_quantiles)
        targets_horizon = output_sequences[:, self.horizon_start:, 0] # Target values from horizon to end of sequence. Shape(N, output_length - horizon_start)
        loss = self.loss.loss(preds_horizon, targets_horizon) # Quantile losses for each batch & timestep. Shape (N, output_length - horizon_start, n_quantiles)

        # Reduce the quantile loss
        loss_reduced = loss.mean(dim = 2) # Average over quantiles. Yields (N, output_length - horizon_start)
        loss_reduced = loss_reduced.sum(dim = 1) # Sum over forecast steps. Yields (N)
        loss_reduced = loss_reduced.mean() # Average over batches. Yields scalar loss for backpropagation.

        # Log the val. loss
        self.log("val_loss", loss_reduced, on_step = True, on_epoch = True, prog_bar = True, logger = False)

        return loss_reduced

    # Define prediction_step
    def predict_step(self, batch, batch_idx):

        # Initialize variables to record horizon, hidden & cell states, predictions
        h = 0
        prev_hiddens = []
        prev_cells = []
        batch_preds = []

        # Get inputs & outputs for first forecast step
        input_sequences, output_sequences = batch
        input_seq = input_sequences # Inputs of the forecast step 0. (N, input_length, input_size) 
        output_seq = output_sequences[:, 0, 1:] # Future covars of forecast step 0. Needed for later forecast steps. (N, input_size - 1)

        # Perform prediction & recording for first forecast step
        # If a hidden & cell state is retained from training, use it.
        if self._final_hiddens_train == None:
            last_hidden_states, last_cell_states, preds = self.forward(input_seq)
        else:
            last_hidden_states, last_cell_states, preds = self.forward(
                input_seq, 
                prev_states = (self._final_hiddens_train, self._final_cells_train)
            )

        prev_hiddens.append(last_hidden_states) # 1-dimensional list. Each element has shape (num_layers, N, hidden_size)
        prev_cells.append(last_cell_states) # 1-dimensional list. Each element has shape (num_layers, N, hidden_size)
        batch_preds.append(preds) # 1-dimensional list. Each element has shape (N, n_quantiles)
        h += 1

        # Perform prediction & recording for remaining forecast steps
        while h < self.output_length:

            # Get inputs & outputs for forecast step h:
            output_seq = torch.cat((
                batch_preds[h-1][:, self._median_quantile_idx].unsqueeze(1), # Point prediction of forecast step 0. (N, 1)
                output_seq # Future covars of forecast step 0. (N, input_size - 1)
            ), dim = 1)
            
            input_seq = torch.cat((
                input_seq[:, 1:, :], # Inputs of the previous forecast step, with the first row dropped. (N, input_length - 1, input_size)
                output_seq.unsqueeze(1) # Prediction & future covars of previous forecast step, the last row of the new input. (N, 1, input_size)
            ), dim = 1)
            
            output_seq = output_sequences[:, h, 1:] # Future covars. of forecast step h. Needed for later forecast steps. (N, input_size-1)

            # Perform training & recording for forecast step h:
            last_hidden_states, last_cell_states, preds = self.forward(
                input_seq, 
                prev_states = (prev_hiddens[h-1], prev_cells[h-1])
            )
            prev_hiddens.append(last_hidden_states)
            prev_cells.append(last_cell_states)
            batch_preds.append(preds)
            h += 1

        # Reshape predictions
        preds = torch.stack(batch_preds, dim = 0) # Yields shape (output_length, N, n_quantiles)
        preds = torch.movedim(preds, 1, 0) # Yields shape (N, output_length, n_quantiles)

        return preds

    # Define optimizer & learning rate scheduler
    def configure_optimizers(self):

        # Adam optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
        
        # Exponential LR scheduler
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(
          optimizer, gamma = self.lr_decay) 
        
        return {
        "optimizer": optimizer,
        "lr_scheduler": {
          "scheduler": lr_scheduler
          }
        }
        

## Hyperparameter tuning with Optuna

In [ ]:
# Define tuning parameters
tol = 0.002 # Change in MAE to avoid early stopping
patience = 5 # N. of rounds with no improvement before early stopping

In [ ]:
# Define Optuna objective
def objective_lstm(trial):

    # Define search space
    learning_rate = trial.suggest_float("learning_rate", 5e-4, 5e-2) # 0.0005 to 0.05
    lr_decay = trial.suggest_float("lr_decay", 0.9, 1)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    hidden_size = 2 ** trial.suggest_int("hidden_size_pw", 2, 8) # 4 to 128, powers of 2
    dropout_rate = trial.suggest_float("dropout_rate", 0, 0.1)

    # Create hyperparameters dict
    hyperparameters_dict = {
        "output_length": output_length,
        "input_size": input_dims,
        "horizon_start": horizon_start,
        "quantiles": quantiles,
        "learning_rate": learning_rate,
        "lr_decay": lr_decay,
        "num_layers": num_layers,
        "hidden_size": hidden_size,
        "dropout_rate": dropout_rate
    }

    # Create early stop callback
    callback_earlystop = L.pytorch.callbacks.EarlyStopping(
        monitor = "val_loss", 
        mode = "min", 
        min_delta = tol, 
        patience = patience
    )

    # Create pruning callback
    callback_pruner = PyTorchLightningPruningCallback(trial, monitor = "val_loss")

    # Create trainer
    trainer = L.Trainer(
        max_epochs = 50,
        accelerator = "gpu",
        devices = "auto",
        precision = "16-mixed",
        callbacks = [callback_earlystop, callback_pruner],
        enable_model_summary = False,
        logger = False,
        enable_progress_bar = False,
        enable_checkpointing = False
    )

    # Create & train model
    model = StatefulQuantileLSTM(hyperparameters_dict)
    trainer.fit(model, tr_loader, val_loader)

    # Retrieve best val score and n. of epochs
    score = callback_earlystop.best_score.cpu().numpy()
    epoch = trainer.current_epoch - callback_earlystop.wait_count # Starts from 1

    # Report best n. of epochs to study
    trial.set_user_attr("n_epochs", epoch)
  
    return score

In [ ]:
# Create study
study_lstm = optuna.create_study(
  sampler = optuna.samplers.TPESampler(seed = random_seed),
  pruner = optuna.pruners.HyperbandPruner(),
  study_name = "tune_lstm",
  direction = "minimize"
)

In [ ]:
# Optimize study
study_lstm.optimize(objective_lstm, n_trials = 500, show_progress_bar = True)

In [ ]:
# Retrieve and export trials
export_trial_no = "4"
trials_lstm = study_lstm.trials_dataframe().sort_values("value", ascending = True)
trials_lstm.to_csv(f"./OutputData/trials_lstm{export_trial_no}.csv", index = False)

## Model testing

In [ ]:
# Import best trial
import_trial_no = "4"
best_trial_lstm= pd.read_csv(f"./OutputData/trials_lstm{import_trial_no}.csv").iloc[0,]

In [ ]:
best_trial_lstm

In [ ]:
# Create hyperparameters dict
hyperparameters_dict = {
    "output_length": output_length,
    "input_size": input_dims,
    "horizon_start": horizon_start,
    "quantiles": quantiles,
    "learning_rate": best_trial_lstm["params_learning_rate"],
    "lr_decay": best_trial_lstm["params_lr_decay"],
    "num_layers": best_trial_lstm["params_num_layers"],
    "hidden_size": int(2 ** best_trial_lstm["params_hidden_size_pw"]),
    "dropout_rate": best_trial_lstm["params_dropout_rate"]
}

In [ ]:
# Create trainer
trainer = L.Trainer(
    max_epochs = int(best_trial_lstm["user_attrs_n_epochs"]),
    accelerator = "gpu",
    devices = "auto",
    precision = "16-mixed",
    enable_model_summary = True,
    logger = False,
    enable_progress_bar = True,
    enable_checkpointing = False
)

In [ ]:
# Create & train model
model = StatefulQuantileLSTM(hyperparameters_dict)
trainer.fit(model, train_loader)

In [ ]:
# Make predictions on test data & backtransform scaled values
preds_raw = trainer.predict(model, test_loader)
preds_raw = torch.cat(preds_raw, dim = 0).cpu().numpy().astype(np.float32)
preds = scaler_test.backtransform_preds(preds_raw)

In [ ]:
# Combine back prediction sequences with times
test_dates = np.stack([sequence.index for sequence in test_output], axis = 0)
df_preds = pd.DataFrame({
    "time": np.ravel(test_dates),
    "pred_low": np.ravel(preds[:, :, 0]),
    "pred_point": np.ravel(preds[:, :, 1]),
    "pred_high": np.ravel(preds[:, :, 2])
})

In [ ]:
df_preds

In [ ]:
# Combine back testing output sequences with input sequences & times

# Create dataframe from all input sequences
test_output_stacked = np.stack(test_output, axis = 0)
df_test_output = pd.DataFrame({
    "time": np.ravel(test_dates),
    "consumption_MWh": np.ravel(test_output_stacked[:, :, 0])
})
df_test_output["time"] = pd.to_datetime(df_test_output["time"])

# Create dataframe from all input sequences
test_input_dates = np.stack([sequence.index for sequence in test_input], axis = 0)
test_input_stacked = np.stack(test_input, axis = 0)
df_test_input = pd.DataFrame({
    "time": np.ravel(test_input_dates),
    "consumption_MWh" : np.ravel(test_input_stacked[:, :, 0])
})
df_test_input["time"] = pd.to_datetime(df_test_input["time"])

# Concatenate & sort by time
df_test = pd.concat([df_test_output, df_test_input])
df_test = df_test.sort_values("time")

In [ ]:
df_test

In [ ]:
# Predicted vs. actual plot, hourly, entire test period
_ = sns.lineplot(
    data = df_test,
    x = "time",
    y = "consumption_MWh",
    label = "Actual values"
)

_ = sns.lineplot(
    data = df_preds,
    x = "time",
    y = "pred_point",
    label = "Predictions, 95% quantile interval"
)

_ = plt.fill_between(
    x = df_preds.time,
    y1 = df_preds.pred_low,
    y2 = df_preds.pred_high,
    label = "95% prediction interval",
    color = "orange",
    alpha = 0.4
)
_ = plt.title("Model: Stateful LSTM")

In [ ]:
def plot_sequence_preds(preds, input_sequences, output_sequences, sequence_index = 0, model = "Stateful LSTM"):

    # Get n. of sequences
    n_sequences = len(output_sequences)
    
    # Get predictions for selected sequence
    preds_low = preds[sequence_index, :, 0]
    preds_point = preds[sequence_index, :, 1]
    preds_high = preds[sequence_index, :, 2]

    # Get & combine actual outputs, inputs, dates
    date_output = output_sequences[sequence_index].index.to_series()
    output = output_sequences[sequence_index].consumption_MWh.values
    
    date_input = input_sequences[sequence_index].index.to_series()
    input = input_sequences[sequence_index].consumption_lag2.values

    date = pd.concat([date_input, date_output], axis = 0)
    actual = np.concatenate([input, output], axis = 0)

    # Plot
    _ = sns.lineplot(
        x = date,
        y = actual,
        label = "Actual values"
    )

    _ = sns.lineplot(
        x = date_output,
        y = preds_point,
        label = "Predictions"
    )
    
    _ = plt.fill_between(
        x = date_output,
        y1 = preds_low,
        y2 = preds_high,
        color = "orange",
        alpha = 0.4
    )

    _ = plt.title(f"Model: {model},\n Sequence index: {sequence_index} of {n_sequences - 1}")
    _ = plt.xlabel("time")
    _ = plt.ylabel("consumption")

In [ ]:
# Predicted vs. actual plots, zoomed in
indices = [i for i in range(0, len(test_output), len(test_output) // 4)]
indices.append(len(test_output) - 1)
for idx in indices:
    plt.figure()
    plot_sequence_preds(preds, test_input, test_output, sequence_index = idx)

In [ ]:
# Calculate performance metrics: MAE, RMSLE, MAPE, pinball
rounding = 4
pd.DataFrame([
    mape(df_test_output.consumption_MWh, df_preds.pred_point) * 100,
    rmsle(df_test_output.consumption_MWh, df_preds.pred_point),
    mae(df_test_output.consumption_MWh, df_preds.pred_point),
    pinball(df_test_output.consumption_MWh, df_preds.pred_point, alpha = quantiles[0]),
    pinball(df_test_output.consumption_MWh, df_preds.pred_point, alpha = quantiles[1]),
    pinball(df_test_output.consumption_MWh, df_preds.pred_point, alpha = quantiles[-1])
], columns = ["Model: Stateful quantile LSTM"],
index = [ 
    "MAPE, point", 
    "RMSLE, point",
    "MAE, point",
    f"Pinball loss, q: {quantiles[0] * 100}%",
    f"Pinball loss, q: {quantiles[1] * 100}%",
    f"Pinball loss, q: {quantiles[-1] * 100}%"
]).round(rounding)

Keep in mind that for the median quantile, pinball loss equals MAE / 2.